In [7]:
!mkdir aoi
# สร้างโฟลเดอร์ชื่อ aoi
!wget  https://raw.githubusercontent.com/ashokdahal/bigdata-geoai/refs/heads/main/bigdata-excercises/aoi/aoi.geojson  -O aoi/aoi.geojson
# ดาวน์โหลดไฟล์ aoi.geojson จาก URL ที่กำหนด แล้วบันทึกไว้ในโฟลเดอร์ aoi

mkdir: cannot create directory ‘aoi’: File exists
--2025-08-25 03:14:31--  https://raw.githubusercontent.com/ashokdahal/bigdata-geoai/refs/heads/main/bigdata-excercises/aoi/aoi.geojson
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 743 [text/plain]
Saving to: ‘aoi/aoi.geojson’

aoi/aoi.geojson     100%[===================>]     743  --.-KB/s    in 0s      

2025-08-25 03:14:31 (55.8 MB/s) - ‘aoi/aoi.geojson’ saved [743/743]



In [13]:
pip install openeo
# import modal

In [10]:
pip install rasterio

In [11]:
import openeo
# นำเข้าไลบรารี openeo สำหรับเชื่อมต่อและประมวลผลข้อมูลดาวเทียม
import rasterio
# นำเข้าไลบรารี rasterio สำหรับการอ่านและเขียนข้อมูลภาพแรสเตอร์
from rasterio.plot import show
# นำเข้าฟังก์ชัน show จาก rasterio สำหรับแสดงผลภาพแรสเตอร์
import matplotlib.pyplot as plt
# นำเข้า pyplot จาก matplotlib สำหรับการสร้างและแสดงแผนภาพ
import matplotlib.colors
# นำเข้า colors จาก matplotlib สำหรับการจัดการสี
import matplotlib.patches as mpatches
# นำเข้า patches จาก matplotlib สำหรับการสร้างรูปร่าง
import geopandas as gpd
# นำเข้า geopandas สำหรับจัดการข้อมูลเชิงพื้นที่ (GeoDataFrame)
import json
# นำเข้าไลบรารี json สำหรับจัดการข้อมูล JSON
import folium
# นำเข้าไลบรารี folium สำหรับสร้างแผนที่เชิงโต้ตอบ
import scipy
# นำเข้าไลบรารี scipy สำหรับการคำนวณทางวิทยาศาสตร์
import numpy as np
# นำเข้าไลบรารี numpy สำหรับการคำนวณเชิงตัวเลขและการจัดการแถวลำดับ (array)

In [12]:
connection = openeo.connect("openeo.dataspace.copernicus.eu").authenticate_oidc()
# เชื่อมต่อกับเซิร์ฟเวอร์ openeo ของ Copernicus และยืนยันตัวตนด้วย OpenID Connect (OIDC)

Authenticated using refresh token.


In [36]:
# def read_json(filename: str) -> dict:
# กำหนดฟังก์ชันเพื่ออ่านไฟล์ JSON
def read_json(filename: str) -> dict:
    with open(filename) as input:
        field = json.load(input)
    return field
# ฟังก์ชันนี้จะเปิดและอ่านไฟล์ JSON ตามชื่อที่กำหนด (filename) แล้วส่งคืนข้อมูลในรูปแบบ dictionary

# เลือกวัน และ Shape
# เลือกช่วงเวลาและรูปร่างขอบเขต
post_date = ["2018-09-09", "2018-12-09"]
# กำหนดช่วงเวลาหลังเกิดเหตุ (วันที่ 9 กันยายน 2018 ถึง 9 ธันวาคม 2018)
# เข้า https://geojson.io/#map=13.22/13.68355/100.55285 เพื่อ Digitize และ Download เป็น GeoJSON และ นำเข้าใน Google Colab
aoi = read_json("aoi/Mapห้วยขาแข้ง.geojson")
# เรียกใช้ฟังก์ชัน read_json เพื่ออ่านไฟล์ aoi.geojson และกำหนดให้เป็นขอบเขตพื้นที่ที่สนใจ (aoi)

In [38]:
m = folium.Map([15.796608,99.138615], zoom_start=13)
# สร้างแผนที่ด้วยไลบรารี Folium โดยกำหนดจุดกึ่งกลางและระดับการซูมเริ่มต้น
folium.GeoJson(aoi).add_to(m)
# เพิ่มขอบเขตพื้นที่ที่สนใจ (aoi) ลงบนแผนที่
#รันภาพ
# แสดงภาพแผนที่
tile = folium.TileLayer(
      tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
      attr = 'Esri',
      name = 'Esri Satellite',
      overlay = False,
      control = True
      ).add_to(m)
# เพิ่มเลเยอร์แผนที่ดาวเทียมจาก Esri ลงบนแผนที่
m
# แสดงผลแผนที่สุดท้าย

In [39]:
# define a function to identify cloud-free pixels in the available data-cube
# กำหนดฟังก์ชันเพื่อระบุพิกเซลที่ไม่มีเมฆในชุดข้อมูลที่มีอยู่
def getBAP(scl, data, reducer="first"):
    mask = (scl == 3) | (scl == 8) | (scl == 9) | (scl == 10)
    # สร้างหน้ากากเพื่อระบุพิกเซลที่เป็นเมฆ (3=เมฆระดับต่ำ, 8=เมฆระดับกลาง, 9=เมฆระดับสูง, 10=เงาเมฆ)

    # mask is a bit noisy, so we apply smoothening
    # หน้ากากมีความไม่สมบูรณ์เล็กน้อย จึงต้องปรับให้เรียบขึ้น
    # 2D gaussian kernel
    # ใช้แกนเกาส์เซียนแบบ 2 มิติ
    g = scipy.signal.windows.gaussian(11, std=1.6)
    kernel = np.outer(g, g)
    kernel = kernel / kernel.sum()

    # Morphological dilation of mask: convolution + threshold
    # การขยายหน้ากากตามลักษณะรูปทรง: ด้วยการรวมพิกเซลและกำหนดค่าเกณฑ์
    mask = mask.apply_kernel(kernel)
    mask = mask > 0.1

    data_masked = data.mask(mask)
    # ซ่อนข้อมูลพิกเซลที่มีเมฆตาม mask

    # now select Best Available Pixel based on the mask
    # จากนั้นเลือกพิกเซลที่ดีที่สุด (Best Available Pixel) โดยอิงตามmask
    return data_masked.reduce_dimension(reducer=reducer, dimension="t")

In [40]:
# load S2 post collection
# โหลดชุดข้อมูลดาวเทียม S2 หลังเกิดเหตุ
s2post = connection.load_collection(
    "SENTINEL2_L2A",
    temporal_extent=post_date,
    spatial_extent=aoi,
    bands=["B02", "B03", "B04", "B08"],
    max_cloud_cover=90,
)

# โหลดข้อมูลภาพถ่ายดาวเทียม Sentinel-2 L2A จากช่วงเวลาหลังเกิดเหตุ ('post_date')
# ในพื้นที่ ('aoi') ที่กำหนด โดยเลือกใช้เฉพาะแถบคลื่น B02, B03, B04, และ B08
# และอนุญาตให้มีเมฆปกคลุมได้สูงสุด 90%

s2post_scl = connection.load_collection(
    "SENTINEL2_L2A",
    temporal_extent=post_date,
    spatial_extent=aoi,
    bands=["SCL"],
    max_cloud_cover=90,
)

# โหลดข้อมูลภาพถ่ายดาวเทียมชุดเดิมอีกครั้ง แต่คราวนี้เลือกเฉพาะแถบคลื่น SCL (Scene Classification Layer)
# ซึ่งใช้สำหรับจำแนกประเภทพื้นผิว เช่น เมฆ, เงาเมฆ, น้ำ, หรือพืช

s2post = getBAP(s2post_scl, s2post, reducer="median")
# ใช้ฟังก์ชัน getBAP ที่คุณสร้างไว้ก่อนหน้า
# เพื่อเลือกพิกเซลที่ดีที่สุด (Best Available Pixel) ที่ไม่มีเมฆจากข้อมูลภาพถ่าย ('s2post')
# โดยใช้ข้อมูล SCL ('s2post_scl') เพื่อระบุและกำจัดเมฆออก
# การใช้ 'reducer="median"' หมายถึง การนำค่ามัธยฐาน (ค่ากลาง) ของพิกเซลที่ดีที่สุดมาใช้

# calculate post ndvi mosaic
# คำนวณค่า NDVI ของภาพหลังเกิดเหตุ
ndvi_post = s2post.ndvi()
# คำนวณค่าดัชนีพืชพรรณ NDVI (Normalized Difference Vegetation Index) จากข้อมูลที่ผ่านการกรองเมฆแล้ว

s2post = s2post.merge_cubes(ndvi_post)

# รวมข้อมูลค่า NDVI เข้ากับชุดข้อมูลภาพถ่าย S2
# ทำให้ชุดข้อมูล 's2post' ที่ได้ในขั้นตอนนี้มีทั้งข้อมูลแถบคลื่นเดิมและค่า NDVI (bands 5)

หาสูตร NDVI จาก LINK https://awesome-ee-spectral-indices.readthedocs.io/en/latest/list.html

หาสูตรอื่นๆ https://open-eo.github.io/openeo-python-client/cookbook/spectral_indices.html

ข้อมูล ดาวเทียม https://openeo.dataspace.copernicus.eu/?discover=0

ค้นหา อัลกอลิทึม https://scikit-learn.org/stable/


In [41]:
# lets execute the process
# เริ่มดำเนินการ
s2post.execute_batch(
    title="Landslides Satellite Image",
    outputfile="Result/Mapห้วยขาแข้ง.tiff"
)
# เรียกใช้การประมวลผลแบบกลุ่ม (batch) ของข้อมูล s2post ที่เตรียมไว้
# โดยกำหนดชื่อผลลัพธ์ว่า "Landslides Satellite Image"
# และบันทึกไฟล์ภาพที่ได้ไว้ที่โฟลเดอร์ Result ในชื่อ "Image.tiff"

0:00:00 Job 'j-250825045608459394421db15a016616': send 'start'
0:00:23 Job 'j-250825045608459394421db15a016616': created (progress 0%)
0:00:29 Job 'j-250825045608459394421db15a016616': created (progress 0%)
0:00:36 Job 'j-250825045608459394421db15a016616': created (progress 0%)
0:00:45 Job 'j-250825045608459394421db15a016616': created (progress 0%)
0:00:56 Job 'j-250825045608459394421db15a016616': created (progress 0%)
0:01:09 Job 'j-250825045608459394421db15a016616': running (progress N/A)
0:01:25 Job 'j-250825045608459394421db15a016616': running (progress N/A)
0:01:45 Job 'j-250825045608459394421db15a016616': running (progress N/A)
0:02:10 Job 'j-250825045608459394421db15a016616': running (progress N/A)
0:02:41 Job 'j-250825045608459394421db15a016616': running (progress N/A)
0:03:20 Job 'j-250825045608459394421db15a016616': running (progress N/A)
0:04:07 Job 'j-250825045608459394421db15a016616': running (progress N/A)
0:05:07 Job 'j-250825045608459394421db15a016616': running (progres

<BatchJob job_id='j-250825045608459394421db15a016616'>